In [27]:
import numpy as np
import pandas as pd
import requests
from datetime import datetime
import json

#Set Display options
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width' , 400)

In [28]:
# Variables
season = "2020-21"

In [29]:
all_players = pd.read_csv (r'./data/' + season + '/Cleaned_data/all_players.csv', parse_dates= ["news_added"])
all_teams = pd.read_csv (r'./data/' + season + '/Cleaned_data/all_teams.csv')
all_events = pd.read_csv (r'./data/' + season + '/Cleaned_data/all_events.csv', parse_dates= ["deadline_time"])
player_types = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_types.csv')
game_phases = pd.read_csv(r'./data/' + season + '/Cleaned_data/game_phases.csv')

player_season_history = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_season_history.csv')
player_gameweek_history = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_gameweek_history.csv' , parse_dates = ['kickoff_time'])
player_future_fixture = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_future_fixture.csv', parse_dates = ["kickoff_time"])




In [30]:
all_teams.head()

,team_code,draw,team_form,team_id,loss,team_name,played,points,position,team_short_name,...,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,NaN,1,0,Arsenal,0,0,0,ARS,...,NaN,False,0,1240,1250,1160,1210,1190,1230,1
1,7,0,NaN,2,0,Aston Villa,0,0,0,AVL,...,NaN,False,0,1130,1150,1140,1170,1120,1160,2
2,36,0,NaN,3,0,Brighton,0,0,0,BHA,...,NaN,False,0,1100,1120,1110,1130,1100,1120,131
3,90,0,NaN,4,0,Burnley,0,0,0,BUR,...,NaN,False,0,1060,1100,1130,1150,1010,1020,43
4,8,0,NaN,5,0,Chelsea,0,0,0,CHE,...,NaN,False,0,1250,1280,1180,1250,1230,1260,4


In [31]:
all_players.head()

,chance_of_playing_next_round,chance_of_playing_this_round,player_code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,player_type_id,ep_next,...,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,PP90,PPMM,VAPM
0,0.0,0.0,37605,0,0,-2,2,0,3,0.0,...,237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.294118
1,0.0,0.0,39476,0,0,-1,1,0,2,0.0,...,205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.408163
2,100.0,50.0,41270,0,0,0,0,0,2,0.8,...,82,NaN,NaN,4.0,NaN,NaN,NaN,1.974922,0.254545,-0.109091
3,100.0,100.0,54694,0,0,-3,3,0,3,4.5,...,22,NaN,NaN,6.0,NaN,1.0,NaN,4.025559,0.341880,0.170940
4,100.0,100.0,58822,0,0,-3,3,0,2,0.5,...,198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-0.425532


In [32]:
player_future_fixture.head()

,fixture_id,fixture_code,home_team_id,team_h_score,away_team_id,team_a_score,event,finished,minutes,provisional_start_time,kickoff_time,event_name,is_home,difficulty,player_id,opponent_team_id,team_id
0,69,2128356,1,NaN,2,NaN,8.0,False,0,False,2020-11-08 19:15:00,Gameweek 8,True,3,1,2,1
1,82,2128369,10,NaN,1,NaN,9.0,False,0,False,2020-11-22 16:30:00,Gameweek 9,False,3,1,10,1
2,89,2128376,1,NaN,20,NaN,10.0,False,0,False,2020-11-28 15:00:00,Gameweek 10,True,3,1,20,1
3,106,2128393,17,NaN,1,NaN,11.0,False,0,False,2020-12-05 15:00:00,Gameweek 11,False,4,1,17,1
4,109,2128396,1,NaN,4,NaN,12.0,False,0,False,2020-12-13 15:00:00,Gameweek 12,True,2,1,4,1


# Merge to all_players

In [33]:
df_merge = all_teams[["team_code" , "team_strength" , "team_short_name" , "strength_overall_home" , "strength_overall_away"]]
all_players = all_players.merge(df_merge, on = 'team_code')

df_merge = player_types[['player_type_id' , 'position_name_short']]
all_players = all_players.merge(df_merge , on = "player_type_id")


In [34]:
all_players.head()

,chance_of_playing_next_round,chance_of_playing_this_round,player_code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,player_type_id,ep_next,...,penalties_order,penalties_text,PP90,PPMM,VAPM,team_strength,team_short_name,strength_overall_home,strength_overall_away,position_name_short
0,0.0,0.0,37605,0,0,-2,2,0,3,0.0,...,NaN,NaN,NaN,0.000000,-0.294118,4,ARS,1240,1250,MID
1,100.0,100.0,54694,0,0,-3,3,0,3,4.5,...,1.0,NaN,4.025559,0.341880,0.170940,4,ARS,1240,1250,MID
2,NaN,NaN,84450,-1,1,-2,2,0,3,1.5,...,NaN,NaN,2.211302,0.320755,-0.056604,4,ARS,1240,1250,MID
3,NaN,NaN,195735,0,0,-3,3,0,3,1.5,...,2.0,NaN,5.346535,0.259740,0.000000,4,ARS,1240,1250,MID
4,0.0,0.0,198849,0,0,-3,3,0,3,0.0,...,NaN,NaN,NaN,0.000000,-0.425532,4,ARS,1240,1250,MID


# Merge to player_gameweek_history

In [38]:
for i in all_players.columns:
    print(i)

chance_of_playing_next_round
chance_of_playing_this_round
player_code
cost_change_event
cost_change_event_fall
cost_change_start
cost_change_start_fall
dreamteam_count
player_type_id
ep_next
ep_this
event_points
first_name
form
player_id
in_dreamteam
news
news_added
now_cost
photo
PPG
second_name
selected_by_percent
status
team_id
team_code
total_points
transfers_in
transfers_in_event
transfers_out
transfers_out_event
value_form
value_season
web_name
total_minutes
goals_scored
assists
clean_sheets
goals_conceded
own_goals
penalties_saved
penalties_missed
yellow_cards
red_cards
saves
bonus
bps
influence
creativity
threat
ict_index
influence_rank
influence_rank_type
creativity_rank
creativity_rank_type
threat_rank
threat_rank_type
ict_index_rank
ict_index_rank_type
corners_and_indirect_freekicks_order
corners_and_indirect_freekicks_text
direct_freekicks_order
direct_freekicks_text
penalties_order
penalties_text
PP90
PPMM
VAPM
team_strength
team_short_name
strength_overall_home
strength_o

In [43]:
player_gameweek_history.head()

,player_id,fixture_id,opponent_team_id,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,...,bps,influence,creativity,threat,ict_index,value,transfers_balance,selected,transfers_in,transfers_out
0,1,2,8,0,False,2020-09-12 11:30:00,0,3,1,0,...,0,0.0,0.0,0.0,0.0,70,0,76656,0,0
1,1,9,19,0,True,2020-09-19 19:00:00,2,1,2,0,...,0,0.0,0.0,0.0,0.0,69,-16828,68335,995,17823
2,1,23,11,0,False,2020-09-28 19:00:00,3,1,3,0,...,0,0.0,0.0,0.0,0.0,69,-11451,59793,675,12126
3,1,29,15,0,True,2020-10-04 13:00:00,2,1,4,0,...,0,0.0,0.0,0.0,0.0,68,-5324,56403,647,5971
4,1,44,12,0,False,2020-10-17 16:30:00,1,0,5,0,...,0,0.0,0.0,0.0,0.0,68,-4224,53689,616,4840


In [44]:
df_merge = all_players[["player_id" , "player_type_id" , "PPG" , "PP90" , "team_strength" , "team_id"]]
player_gameweek_history = player_gameweek_history.merge(df_merge , on = "player_id")

player_gameweek_history.head()

,player_id,fixture_id,opponent_team_id,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,...,value,transfers_balance,selected,transfers_in,transfers_out,player_type_id,PPG,PP90,team_strength,team_id
0,1,2,8,0,False,2020-09-12 11:30:00,0,3,1,0,...,70,0,76656,0,0,3,0.0,NaN,4,1
1,1,9,19,0,True,2020-09-19 19:00:00,2,1,2,0,...,69,-16828,68335,995,17823,3,0.0,NaN,4,1
2,1,23,11,0,False,2020-09-28 19:00:00,3,1,3,0,...,69,-11451,59793,675,12126,3,0.0,NaN,4,1
3,1,29,15,0,True,2020-10-04 13:00:00,2,1,4,0,...,68,-5324,56403,647,5971,3,0.0,NaN,4,1
4,1,44,12,0,False,2020-10-17 16:30:00,1,0,5,0,...,68,-4224,53689,616,4840,3,0.0,NaN,4,1


## Save to file

In [45]:
all_players.to_csv (r'./data/' + season + '/Merged_data/all_players_merged.csv', index = False, header=True)
player_gameweek_history.to_csv(r'./data/' + season + '/Merged_data/player_gameweek_history.csv' , index = False, header=True)
